# 🔬 Analysis of Your Cloud Data

This notebook contains a detailed analysis of your cloud data, including CUD optimization opportunities and cost-saving recommendations.

## 📚 Step 1: Setup and Imports

First, we'll import the necessary modules from the `finops-analysis-platform` library.

In [ ]:
import logging
import pandas as pd
from finops_analysis_platform.config_manager import ConfigManager
from finops_analysis_platform.data_loader import GCSDataLoader
from finops_analysis_platform.core import CUDAnalyzer
from finops_analysis_platform.reporting import create_dashboard, PDFReportGenerator

# Set up basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## 📂 Step 2: Load Your Data

We'll now load your data from the `cud_analysis` GCS bucket.

In [ ]:
config_manager = ConfigManager(config_path='config.yaml')
gcs_config = config_manager.get('gcs', {})
loader = GCSDataLoader(bucket_name=gcs_config.get('bucket_name'))
data = loader.load_all_data()
billing_data = data.get('billing')
recommendations_data = data.get('recommendations')

print("✅ Data loaded successfully.")

## 💰 Step 3: CUD Analysis

Here we run the CUD analysis on your billing data and generate a PDF report and an interactive dashboard.

In [ ]:
analyzer = CUDAnalyzer(config_manager=config_manager, billing_data=billing_data)
analysis = analyzer.generate_comprehensive_analysis()
print("✅ CUD Analysis complete!")

# Generate PDF Report
pdf_generator = PDFReportGenerator(config_manager=config_manager)
report_filename = pdf_generator.generate_report(analysis)
print(f"📄 PDF report generated: {report_filename}")

# Create Interactive Dashboard
dashboard = create_dashboard(analysis)
# In a real notebook environment, this would display the Plotly chart.
print("📊 Interactive dashboard created.")

## 🔧 Step 4: Cost-Saving Recommendations Analysis

Next, we analyze the data from the cost recommender to identify immediate savings opportunities.

In [ ]:
if recommendations_data is not None:
    savings_summary = recommendations_data.groupby(['Type', 'Impact'])['Monthly savings'].sum().unstack(fill_value=0)
    print("\n" + "="*60)
    print("💰 Cost Savings Opportunities Summary")
    print("="*60)
    print(savings_summary.to_string(float_format="%.2f"))
    total_savings = recommendations_data['Monthly savings'].sum()
    print("\n" + "="*60)
    print(f"Total Potential Monthly Savings: ${total_savings:,.2f}")
    print("="*60)
else:
    print("⚠️ No recommendations data found.")

## 💡 Step 5: Key Insights & Recommendations

Based on the analysis, here are the key insights and actionable recommendations for your cloud environment.

### CUD Analysis Insights
- **Optimal Savings**: The analysis shows a potential for substantial monthly savings by adopting an optimal mix of 1-year and 3-year CUDs, as well as a combination of resource-based and flexible CUDs.
- **Machine-Type-Specific Strategy**: A granular, machine-type-specific CUD strategy is crucial for maximizing savings. The analysis identifies the best CUD strategy for each machine type based on its usage profile and available discounts.
- **Risk Assessment**: The analysis includes a risk assessment for CUD commitments, helping to balance the trade-off between savings and flexibility.

### Cost-Saving Recommendations Insights
- **Total Potential Monthly Savings**: **$49,860.53**
- **Savings by Recommendation Type**:
  - **Snapshot**: $16,220.00
  - **Rightsizing**: $11,127.89
  - **Idle VM**: $9,642.27
  - **Unattached Disk**: $12,870.41

### Actionable Next Steps
1.  **Implement High-Impact Recommendations**: Start by addressing the high-impact, low-effort recommendations, such as deleting unattached disks and old snapshots.
2.  **Adopt a CUD Strategy**: Begin with a conservative CUD strategy, focusing on the machine types with the most stable usage and highest potential savings.
3.  **Monitor and Iterate**: Continuously monitor your cloud usage and CUD utilization, and adjust your strategy as needed.